In [29]:
import torch
from transformers import AutoTokenizer, AutoModel

model_name = "vinai/phobert-base"

# Tokenizer hỗ trợ progress_bar
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, progress_bar=False)

# Model **không có progress_bar**
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
try: 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"Model đang chạy trên thiết bị: {device}")
except Exception as e:
    print(f"Cảnh báo: Không thể chuyển model sang GPU: {e}")
    model.to("cpu")


Model đang chạy trên thiết bị: cuda


In [31]:
import psycopg2
from psycopg2 import sql

conn = psycopg2.connect(
    host="localhost", # Bắt buộc kết nối qua mạng TCP/IP
    dbname="postgres", 
    user="postgres", 
    password="khangduong"
)

cur = conn.cursor()


In [ ]:
cur.execute("drop table if exists dbvector;")

In [32]:
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
cur.execute("CREATE TABLE IF NOT EXISTS dbvector1(id SERIAL PRIMARY KEY, EMBEDDING VECTOR(768), document varchar(20));")

In [13]:
conn.rollback()

In [33]:
import os
import re

# Khởi tạo thư mục gốc chứa các file .md
base_dir = "training_output"
# Thay đổi cấu trúc dữ liệu thành dictionary: {folder_name: [list_of_heading_lines]}
extracted_data = {}

print(f"Bắt đầu quét thư mục '{base_dir}'...")

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file == "main.md":
            md_path = os.path.join(root, file)

            # Lấy tên thư mục (folder name) làm key
            # Ví dụ: nếu root là 'training_output/project_a', folder_name sẽ là 'project_a'
            folder_name = os.path.basename(root)
            
            try:
                with open(md_path, "r", encoding="utf-8") as f:
                    lines = f.readlines()
            except FileNotFoundError:
                print(f"Lỗi: Không tìm thấy file tại {md_path}")
                continue
            except Exception as e:
                print(f"Lỗi khi đọc file {md_path}: {e}")
                continue
            
            # --- Logic lọc đã được sửa đổi ---
            # CHỈ lấy các dòng bắt đầu bằng một hoặc nhiều dấu # (headings)
            heading_lines = []
            for line in lines:
                stripped_line = line.strip()
                
                # Kiểm tra xem dòng có bắt đầu bằng '#' và theo sau là một khoảng trắng không.
                # Đã loại bỏ điều kiện kiểm tra dòng bullet list (-)
                if re.match(r"^#+\s", stripped_line):
                    heading_lines.append(stripped_line)
            
            # Lưu trữ dữ liệu vào dictionary
            if heading_lines:
                extracted_data[folder_name] = heading_lines

print(f"✅ Quá trình trích xuất hoàn tất.")
print(f"Tổng cộng có {len(extracted_data)} thư mục đã được xử lý.")

# In ra ví dụ để kiểm tra cấu trúc dictionary
if extracted_data:
    first_key = list(extracted_data.keys())[0]
    print("\n--- Cấu trúc dữ liệu (Ví dụ) ---")
    print(f"Key (Tên thư mục): '{first_key}'")
    print(f"Value (Các dòng Heading):")
    for line in extracted_data[first_key]:
        print(f"  - {line}")
else:
    print("Không tìm thấy dữ liệu hoặc file 'main.md' nào chứa dòng heading.")


Bắt đầu quét thư mục 'training_output'...
✅ Quá trình trích xuất hoàn tất.
Tổng cộng có 100 thư mục đã được xử lý.

--- Cấu trúc dữ liệu (Ví dụ) ---
Key (Tên thư mục): 'Public001'
Value (Các dòng Heading):
  - #  Public001.pdf
  - # Giới thiệu
  - # Nhà thông minh trước năm 2010
  - ### Đo điều kiện nhà
  - ### Quản lý thiết bị gia dụng
  - ### Kiểm soát việc ra vào nhà
  - # Tổng quan về Internet vạn vật
  - # Điện toán đám mây và đóng góp của nó cho IoT và nhà thông minh
  - # Nhà thông minh từ sau năm 2010
  - # Các khía cạnh thực tế và cân nhắc triển khai cho IoT và nhà thông minh
  - # Ví dụ về nhà thông minh và IoT
  - # Kết luận


In [19]:
for document_name, doc_lines in enumerate(extracted_data.items()):
    print(doc_lines)

('Public001', ['#  Public001.pdf', '# Giới thiệu', '# Nhà thông minh trước năm 2010', '### Đo điều kiện nhà', '### Quản lý thiết bị gia dụng', '### Kiểm soát việc ra vào nhà', '# Tổng quan về Internet vạn vật', '# Điện toán đám mây và đóng góp của nó cho IoT và nhà thông minh', '# Nhà thông minh từ sau năm 2010', '# Các khía cạnh thực tế và cân nhắc triển khai cho IoT và nhà thông minh', '# Ví dụ về nhà thông minh và IoT', '# Kết luận'])
('Public002', ['#  Public002.pdf', '# Bộ Môn Hệ Thống Thông Tin', '# Bộ Môn Mạng Máy Tính'])
('Public003', ['#  Public003.pdf', '# GIỚI THIỆU', '# CÁC NGHIÊN CỨU LIÊN QUAN', '# PHƯƠNG PHÁP ĐỀ XUẤT', '# THỰC NGHIỆM VÀ KẾT QUẢ', '# câu tiếng Việt.', '# KẾT LUẬN'])
('Public004', ['#  Public004.pdf', '# GIỚI THIỆU', '# TỔNG QUAN', '# CÁC CÔNG NGHỆ IN BÊ TÔNG 3D', '### * Công nghệ Contour Crafting', '### * Công nghệ Concrete Printing', '### * Công nghệ Concrete On-Site 3D Printing', '### * Công nghệ in bê tông 3D quy mô lớn sử dụng bê tông cường độ cao', '#

In [34]:
import torch
# Cần import thư viện database (ví dụ: psycopg2 cho PostgreSQL)
# import psycopg2
# from psycopg2 import sql

# Lưu ý: Các biến 'tokenizer', 'model', 'device', 'extracted_data'
# được giả định là đã được định nghĩa và khởi tạo trong môi trường của bạn.

# --- 1. Khởi tạo và Tạo Embedding (Tách biệt theo từng dòng) ---
# Danh sách để lưu trữ TẤT CẢ các bản ghi (record), mỗi record là một dòng/segment
all_embedding_records = [] 

# Lặp trực tiếp qua key (tên document) và value (list các dòng Heading) của extracted_data
for document_name, doc_lines in extracted_data.items():
    
    print(f"--- Đang xử lý Document: {document_name} ({len(doc_lines)} segments) ---")
    
    # Lặp qua từng dòng văn bản (segment) trong tài liệu này
    for idx, segment_text in enumerate(doc_lines):
        
        # Bỏ qua các dòng trống (mặc dù đã được xử lý ở bước trước, vẫn giữ để phòng hờ)
        if not segment_text.strip():
            continue

        try:
            # BƯỚC TIỀN XỬ LÝ QUAN TRỌNG: XÓA TẤT CẢ DẤU CÁCH TRƯỚC KHI TOKEN HÓA
            preprocessed_text = segment_text.replace(' ', '')
            
            # 1. Token hóa và chuyển sang PyTorch Tensor cho segment hiện tại
            # Sử dụng văn bản đã được tiền xử lý
            inputs = tokenizer(preprocessed_text, return_tensors="pt", truncation=True, max_length=756)
            
            # 2. Chuyển inputs lên thiết bị GPU (device)
            inputs = {key: value.to(device) for key, value in inputs.items()}
            
            with torch.no_grad():
                # 3. Chạy mô hình để lấy outputs
                outputs = model(**inputs)
                
                # 4. Trích xuất embedding của token [CLS]
                embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

            # Thêm record vào danh sách
            all_embedding_records.append({
                "document": document_name, # Key của dictionary -> document column
                "embedding": embedding.tolist() # Vector đã tạo -> EMBEDDING column
            })
            
            if idx % 50 == 0:
                 print(f"✅ Đã tạo embedding cho dòng {idx} trong document: {document_name}")
        
        except RuntimeError as cuda_err:
             if 'CUDA error: device-side assert triggered' in str(cuda_err):
                 print(f"--- LỖI CUDA (Bỏ qua segment) ---")
                 print(f"Document: {document_name}, Segment Index: {idx}")
                 print(f"Nội dung đã xử lý gây lỗi: {preprocessed_text[:100]}...")
                 continue
             else:
                 raise # Ném các lỗi RuntimeError khác
        
        except Exception as e:
            print(f"❌ Lỗi không xác định khi xử lý segment {idx} của {document_name}: {e}")
            continue


# In ra kết quả xử lý
print("\n--- Kết quả xử lý Embedding ---")
print(f"Tổng số segments (dòng) đã được nhúng: {len(all_embedding_records)}")

# --- 2. Kết nối và Chèn Dữ liệu vào Database (Chèn theo từng dòng/segment) ---

# Giả định rằng conn, cur, và sql đã được định nghĩa và kết nối.
try:
    print("\n--- Bắt đầu chèn dữ liệu vào Database ---")
    
    # Lặp qua danh sách tất cả các records đã tạo (mỗi record là 1 dòng)
    for record in all_embedding_records:
        # Chèn embedding vào bảng dbvector
        cur.execute(
            sql.SQL("INSERT INTO dbvector1 (EMBEDDING, document) VALUES (%s, %s)"),
            (record['embedding'], record['document'])
        )
    
    # Commit giao dịch một lần duy nhất để tối ưu hiệu suất
    conn.commit()
    print(f"✅ HOÀN TẤT: {len(all_embedding_records)} embeddings đã được lưu trữ vào database.")
    
except NameError:
    print("🚨 LỖI KẾT NỐI: Vui lòng định nghĩa và thiết lập kết nối Database (conn, cur, sql) trước khi chạy phần chèn dữ liệu.")
except Exception as e:
    # conn.rollback() # Cần đảm bảo conn đã được định nghĩa
    print(f"❌ LỖI TRONG QUÁ TRÌNH CHÈN DỮ LIỆU: {e}. Vui lòng kiểm tra lại kết nối và lược đồ bảng.")

# !!! Quan trọng: Đừng quên đóng kết nối Database khi kết thúc chương trình !!!


--- Đang xử lý Document: Public001 (12 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public001
--- Đang xử lý Document: Public002 (3 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public002
--- Đang xử lý Document: Public003 (7 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public003
--- Đang xử lý Document: Public004 (12 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public004
--- Đang xử lý Document: Public005 (17 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public005
--- Đang xử lý Document: Public006 (9 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public006
--- Đang xử lý Document: Public007 (5 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public007
--- Đang xử lý Document: Public008 (40 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public008
--- Đang xử lý Document: Public009 (2 segments) ---
✅ Đã tạo embedding cho dòng 0 trong document: Public009
--- Đang xử lý Document:

In [38]:
cur.close()
conn.close()